In [1]:
import pandas as pd
import seaborn as sns
from ipyleaflet import *
from branca.colormap import linear
import matplotlib.pyplot as plt
import json
import pandas
import folium
import geopandas as gpd
from colour import Color
from pandas_profiling import ProfileReport
import numpy as np
from ipywidgets import *
import sklearn as skl
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
# On importe le data frame que l'on a construit (cf note book get_data)

#df_regression = pd.read_csv(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/df_regression.csv')
df_regression = pd.read_csv('../df_regression.csv')


#On ne garde que les données à partir de 1995 pour que la régression soit la plus fiable possible
df_regression = df_regression[df_regression['year']>= 1995]

In [3]:
# On recalibre les données pour effectuer une régression, passage du format long au format wide
df_final = pd.DataFrame()
for cntry in df_regression.country.unique():
    df_temp = df_regression[df_regression.country == cntry]
    df_new = pd.DataFrame({cntry:[]})
    columns = np.array(df_temp.columns)
    for y in range(1990,2019):

        if df_temp[df_temp.year == y].empty:

            for col in columns:
                if col != 'year' and col != 'country':
                    filler = df_temp[col].mean()
                    df_new.loc[str(y) + '-' + col] = filler
        else:

            for col in columns:
                if col != 'year' and col != 'country':
                    df_new.loc[str(y) + '-' + col] = df_temp[df_temp.year == y][col].values

    df_final = pd.concat([df_final, df_new.transpose()])
    
df_final

,1990-croissancePIB,1990-mortalite-enf,1990-population,1990-export,1990-polity2,1990-ratio-zone,1990-ratio,1991-croissancePIB,1991-mortalite-enf,1991-population,...,2017-polity2,2017-ratio-zone,2017-ratio,2018-croissancePIB,2018-mortalite-enf,2018-population,2018-export,2018-polity2,2018-ratio-zone,2018-ratio
Zimbabwe,1.268572,20981.083333,1.256234e+07,136.330378,-1.416667,0.000450,0.000000,1.268572,20981.083333,1.256234e+07,...,4.0,0.000030,0.000000,4.824211,17413.0,14438812.0,210.771948,4.0,0.002011,0.000000
Zambia,5.534573,33454.041667,1.268306e+07,494.755586,5.000000,0.006294,0.000008,5.534573,33454.041667,1.268306e+07,...,6.0,0.000293,0.000000,4.034494,26866.0,17351714.0,1012.443792,6.0,0.002291,0.000000
Yemen,1.120798,40852.250000,2.128825e+07,111.755404,-1.166667,0.000316,0.000000,1.120798,40852.250000,2.128825e+07,...,0.0,0.000040,0.000000,0.752447,37517.0,28498683.0,33.288874,0.0,0.000128,0.000000
Vietnam,6.671504,31087.041667,8.523303e+07,500.452668,-7.000000,0.000000,0.000000,6.671504,31087.041667,8.523303e+07,...,-7.0,0.000000,0.000000,7.075789,25987.0,95545959.0,1682.653459,-7.0,0.000000,0.000000
Uzbekistan,5.771034,22790.041667,2.733835e+07,255.060565,-9.000000,0.132058,0.000215,5.771034,22790.041667,2.733835e+07,...,-9.0,0.199614,0.000000,5.354997,11479.0,32956100.0,387.671272,-9.0,0.312684,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Argentina,2.205179,10587.208333,3.954242e+07,180.912015,8.000000,0.003082,0.000000,2.205179,10587.208333,3.954242e+07,...,9.0,0.028273,0.000000,-2.617396,6560.0,44494502.0,234.543108,9.0,0.027312,0.000000
Angola,6.357488,82248.166667,2.124495e+07,381.830277,-2.958333,0.000045,0.006081,6.357488,82248.166667,2.124495e+07,...,-2.0,0.000000,0.000262,-2.003630,64019.0,30809787.0,514.553299,-2.0,0.000000,0.000279
Algeria,3.416667,21136.541667,3.457893e+07,175.686146,0.125000,0.010172,0.014457,3.416667,21136.541667,3.457893e+07,...,2.0,0.019693,0.000777,1.100000,20875.0,42228415.0,189.716555,2.0,0.024777,0.000407
Albania,4.837915,812.416667,2.994021e+06,435.138961,7.375000,0.000000,0.000003,4.837915,812.416667,2.994021e+06,...,9.0,0.000000,0.000000,4.019360,289.0,2866376.0,1097.548787,9.0,0.000000,0.000000


In [4]:
# On va essayer de prédire le ratio de risque de 2018 en utilisant toutes les données antérieures
df_final.rename(columns = {'2018-ratio':'aim'}, inplace = True)
df_final

# On sépare le jeu de données en entrainement / test
train, test = train_test_split(df_final, test_size = 0.2)
X_train = train.iloc[:, :-2]
y_train = train.iloc[:,-1:]
X_test = test.iloc[:, :-2]
y_test = test.iloc[:,-1:]

# On standardise les valeurs passées en entrée
scalerX = StandardScaler().fit(X_train)
scalerX_test = StandardScaler().fit(X_test)

X_strain = scalerX.transform(X_train)
X_stest = scalerX_test.transform(X_test)

In [5]:
# On effectue une régression linéaire utilisant toutes les données depuis 1995 pour prédire la dangerosité du pays en 2018

clf = LinearRegression()
clf.fit(X_strain, y_train.values)

# On regarde les prédictions
X_test['aim'] = y_test
X_test['prediction'] = clf.predict(X_stest)

In [6]:
X_test['country'] = X_test.index.values
X_test

,1990-croissancePIB,1990-mortalite-enf,1990-population,1990-export,1990-polity2,1990-ratio-zone,1990-ratio,1991-croissancePIB,1991-mortalite-enf,1991-population,...,2017-ratio-zone,2017-ratio,2018-croissancePIB,2018-mortalite-enf,2018-population,2018-export,2018-polity2,aim,prediction,country
Jordan,4.361896,3522.708333,6.720999e+06,268.513723,-2.500000,0.220246,1.385984e-04,4.361896,3522.708333,6.720999e+06,...,0.453533,0.000000,1.934092,2986.000000,9.965322e+06,408.091040,-3.000000,0.000000,-0.004019,Jordan
Colombia,3.336437,14700.125000,4.317453e+07,218.455278,7.000000,0.003824,2.442160e-02,3.336437,14700.125000,4.317453e+07,...,0.028343,0.000343,2.564324,8984.000000,4.966106e+07,320.271331,7.000000,0.002814,-0.096558,Colombia
Albania,4.837915,812.416667,2.994021e+06,435.138961,7.375000,0.000000,2.531722e-06,4.837915,812.416667,2.994021e+06,...,0.000000,0.000000,4.019360,289.000000,2.866376e+06,1097.548787,9.000000,0.000000,0.040488,Albania
Togo,4.123253,13612.791667,5.932930e+06,212.480952,-2.416667,0.000426,9.235819e-04,4.123253,13612.791667,5.932930e+06,...,0.000545,0.000000,4.976213,12122.000000,7.889095e+06,296.543113,-2.000000,0.000000,-0.021874,Togo
Philippines,5.076581,60384.833333,8.844496e+07,116.178024,8.000000,0.000000,1.176601e-02,5.076581,60384.833333,8.844496e+07,...,0.000000,0.017240,6.341486,48672.000000,1.066514e+08,182.013159,8.000000,0.005069,-0.015009,Philippines
Syria,0.385024,9456.291667,1.796826e+07,130.397902,-7.791667,0.090495,1.446577e-01,0.385024,9456.291667,1.796826e+07,...,0.122851,0.338172,1.475766,7725.000000,1.694506e+07,43.168321,-9.000000,0.254909,0.349312,Syria
Uruguay,2.734467,603.958333,3.343552e+06,215.215527,10.000000,0.003058,0.000000e+00,2.734467,603.958333,3.343552e+06,...,0.028273,0.000000,0.477907,309.000000,3.449290e+06,326.733124,10.000000,0.000000,-0.011714,Uruguay
Switzerland,1.951016,337.416667,7.640861e+06,223.484404,10.000000,0.000100,0.000000e+00,1.951016,337.416667,7.640861e+06,...,0.000020,0.000000,2.916905,318.000000,8.514329e+06,386.023344,10.000000,0.000000,0.002079,Switzerland
Finland,2.321504,169.916667,5.300632e+06,134.301746,10.000000,0.000002,0.000000e+00,2.321504,169.916667,5.300632e+06,...,0.000050,0.000000,1.141948,101.000000,5.515525e+06,164.973484,10.000000,0.000000,0.003849,Finland
Lesotho,2.801595,4356.625000,2.012974e+06,291.438925,7.291667,0.000265,5.303871e-05,2.801595,4356.625000,2.012974e+06,...,0.000000,0.000000,-1.223137,3868.000000,2.108327e+06,553.316430,8.000000,0.000000,-0.004112,Lesotho


In [7]:
# On récupére une base de donnée comprenant une liste de données sur les pays du monde
countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

In [9]:
# on crée une carte vierge 
m_predict = folium.Map(tiles='CartoDB Positron',attr=None,zoom_start=10)
# on genère la carte à deux couches  
folium.Choropleth(
  geo_data=countries[countries.name.isin(X_test.index.values)].to_json(),
    data=X_test,
    columns=['country','prediction'],
    key_on="properties.name", 
     fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0,
    smooth_factor=0).add_to(m_predict)

m_aim = folium.Map(tiles='CartoDB Positron',attr=None,zoom_start=4)
# on genère la carte à deux couches  
folium.Choropleth(
  geo_data=countries[countries.name.isin(X_test.index.values)].to_json(),
    data=X_test,
    columns=['country','aim'],
    key_on="properties.name", 
     fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0,
    smooth_factor=0).add_to(m_aim)

In [10]:
m_predict

In [11]:
m_aim